In [1]:
!!pip install contractions --upgrade
!pip install textsearch
!pip install cufflinks --upgrade
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import string
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem import WordNetLemmatizer, PorterStemmer
ps = PorterStemmer()
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
from sklearn.model_selection import train_test_split
from sklearn import svm
import itertools
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
#import cufflinks as cf
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import tqdm
import contractions
from bs4 import BeautifulSoup
import unicodedata
import gensim
#from tensorflow import keras
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dropout, Activation, Dense
from sklearn.preprocessing import LabelEncoder
#from tensorflow.keras.layers import Flatten
#from tensorflow.keras.layers import Conv1D
#from tensorflow.keras.layers import MaxPooling1D
#from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
#from keras.utils.np_utils import to_categorical
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

Requirement already up-to-date: cufflinks in ./anaconda3/envs/newenvt/lib/python3.6/site-packages (0.16)


[nltk_data] Downloading package punkt to /Users/pankaagr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pankaagr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pankaagr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/Users/pankaagr/anaconda3/envs/newenvt/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/pankaagr/anaconda3/envs/newenvt/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/pankaagr/anaconda3/envs/newenvt/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

In [2]:
df = pd.read_csv('train.tsv',sep='\t')

In [3]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
PhraseId      156060 non-null int64
SentenceId    156060 non-null int64
Phrase        156060 non-null object
Sentiment     156060 non-null int64
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


In [5]:
df.isnull().sum()

PhraseId      0
SentenceId    0
Phrase        0
Sentiment     0
dtype: int64

In [6]:
df.Sentiment.value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [7]:
#text Pre-Processing
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: a modified initial string
    """
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ',text)
    text = BAD_SYMBOLS_RE.sub(' ',text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text
df['Phrase'] = df['Phrase'].apply(clean_text)
df['Phrase'] = df['Phrase'].str.replace('\d+','')

In [8]:
#some more text-PreProcessing
from bs4 import BeautifulSoup
import unicodedata
import contractions
def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

Phrase_reviews = df['Phrase'].values
norm_phrases = pre_process_corpus(Phrase_reviews)

  0%|          | 0/156060 [00:00<?, ?it/s]/Users/pankaagr/anaconda3/envs/newenvt/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning:

"b'movies'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.

  1%|          | 1921/156060 [00:00<00:16, 9504.27it/s]/Users/pankaagr/anaconda3/envs/newenvt/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning:

"b'music'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.

  3%|▎         | 4698/156060 [00:00<00:16, 9305.99it/s]/Users/pankaagr/anaconda3/envs/newenvt/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning:

"b'pictures'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.

  4%|▎         | 5604/156060 [00:00<00:16, 9229.87it/s]/Users/pankaagr/anaconda3/envs/newenvt/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning:



In [9]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,series escapades demonstrating adage good goos...,1
1,2,1,series escapades demonstrating adage good goose,2
2,3,1,series,2
3,4,1,,2
4,5,1,series,2


In [10]:
df.isnull().sum()

PhraseId      0
SentenceId    0
Phrase        0
Sentiment     0
dtype: int64

In [11]:
 !pip install "tensorflow>=1.7.0"
 !pip install tensorflow-hub

In [12]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)


Instructions for updating:
Colocations handled automatically by placer.


W1017 01:09:18.972494 4705195456 deprecation.py:323] From /Users/pankaagr/anaconda3/envs/newenvt/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [13]:
#defining elmo vectors

def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [14]:
list_train = [df[i:i+100] for i in range(0,df.shape[0],100)]

In [ ]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['Phrase']) for x in tqdm.tqdm(list_train)]

  0%|          | 0/1561 [00:00<?, ?it/s]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1017 01:09:20.050660 4705195456 saver.py:1483] Saver not created because there are no variables in the graph to restore
  0%|          | 1/1561 [00:06<2:41:24,  6.21s/it]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1017 01:09:26.159042 4705195456 saver.py:1483] Saver not created because there are no variables in the graph to restore
  0%|          | 2/1561 [00:12<2:42:17,  6.25s/it]

...

I1018 23:12:38.012126 4705195456 saver.py:1483] Saver not created because there are no variables in the graph to restore
 27%|██▋       | 417/1561 [46:21:18<427:25:11, 1345.03s/it]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1018 23:31:18.352905 4705195456 saver.py:1483] Saver not created because there are no variables in the graph to restore
 27%|██▋       | 418/1561 [46:41:22<413:40:30, 1302.91s/it]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1018 23:52:04.631125 4705195456 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [ ]:
#concatenating vectors
elmo_train_new = np.concatenate(elmo_train, axis = 0)

In [ ]:
# save elmo_train_new
import pickle
pickle_out = open("elmo_train_sentiment_movie.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

In [ ]:
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  df['Sentiment'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

In [ ]:
preds_valid = lreg.predict(xvalid)


In [ ]:
from sklearn.metrics import accuracy_score

score =accuracy_score(yvalid,preds_valid)

In [ ]:
print(score)